In [1]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

import seaborn as sns
from ionpy.analysis import ResultsLoader
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

# Results loader object does everything
rs = ResultsLoader()
root = "/storage/vbutoi/scratch/ESE"

%load_ext yamlmagic
%load_ext autoreload
%autoreload 2

In [2]:
from ese.experiment.analysis.inference import load_cal_inference_stats

path = f"{root}/11_03_23_WMH_Inference"

pix_inference_dicts, image_inference_dfs, metadata_df = load_cal_inference_stats(
    path,
    return_metadata=True
)

In [3]:
metadata_df['log_set']

0    20231103_105351-RJRW-cec8ed632e9e2042a7912ac41...
Name: log_set, dtype: object

In [4]:
# Get the pixel preds out
# pixel_preds_dict = pix_inference_dicts[metadata_df['log_set'][0]]
image_pred_df = image_inference_dfs[metadata_df['log_set'][0]]

## Pixel-level Analysis

In [5]:
# from ese.experiment.analysis.err_diagrams import viz_accuracy_vs_confidence

# viz_accuracy_vs_confidence(
#     pixel_preds_dict,
#     title="WMH Confidence vs Accuracy per (Bin and Predicted Label)",
#     x="pred_label",
#     hue="measure",
#     col="bin_num",
#     kind="bar",
#     add_average=True,
#     add_proportion=True,
#     facet_kws={'sharey': False, 'sharex': False}
#     )

In [6]:
# from ese.experiment.analysis.err_diagrams import viz_accuracy_vs_confidence

# viz_accuracy_vs_confidence(
#     pixel_preds_dict,
#     title="WMH Confidence vs Accuracy per (Bin and Num Neighbors)",
#     x="num_neighbors",
#     hue="measure",
#     col="bin_num",
#     kind="bar",
#     add_average=True,
#     add_proportion=True,
#     facet_kws={'sharey': False, 'sharex': False},
#     )

## Image-level Analysis

In [7]:
image_pred_df.head()

,accuracy,bin_weighting,cal_metric,cal_score,data_id,dice,w_accuracy,slice_idx,annotator,axis,num_slices,preload,slicing,task,version,split,_class
0,0.000061,proportional,{'ECE': {'func': <cyfunction ECE at 0x7f3b7879...,0.000039,103,0.000122,0.5,0,observer_o12,0,1,False,dense_full,Amsterdam,0.2,cal,ese.experiment.datasets.WMH
1,0.000061,uniform,{'ECE': {'func': <cyfunction ECE at 0x7f3b7879...,0.000004,103,0.000122,0.5,0,observer_o12,0,1,False,dense_full,Amsterdam,0.2,cal,ese.experiment.datasets.WMH
2,0.000061,proportional,{'TL_ECE': {'func': <cyfunction TL_ECE at 0x7f...,0.000039,103,0.000122,0.5,0,observer_o12,0,1,False,dense_full,Amsterdam,0.2,cal,ese.experiment.datasets.WMH
3,0.000061,uniform,{'TL_ECE': {'func': <cyfunction TL_ECE at 0x7f...,0.000004,103,0.000122,0.5,0,observer_o12,0,1,False,dense_full,Amsterdam,0.2,cal,ese.experiment.datasets.WMH
4,0.000061,proportional,{'CW_ECE': {'func': <cyfunction CW_ECE at 0x7f...,0.000039,103,0.000122,0.5,0,observer_o12,0,1,False,dense_full,Amsterdam,0.2,cal,ese.experiment.datasets.WMH


In [10]:
from ese.experiment.analysis.err_diagrams import viz_cal_metric_corr

viz_cal_metric_corr(
    image_pred_df,
    title="WMH Calibration Metric Correlation",
    )

TypeError: unhashable type: 'dict'